In [ ]:
import pygame
from pygame.locals import *
from sys import exit

pygame.init()
clock = pygame.time.Clock()

In [ ]:
WIDTH: int = 640
HEIGHT: int = 480
SCREEN_SIZE: tuple = (WIDTH, HEIGHT)
TITLE: str = "Hello PyGame World!"
FIXED_FPS: int = 60

screen = pygame.display.set_mode(SCREEN_SIZE)
pygame.display.set_caption(TITLE)

In [ ]:
# Other imports
from numpy.linalg import norm
from random import randint


In [ ]:
class InputEvent:

    def get_input_strength() -> list:
        is_pressed = pygame.key.get_pressed()
        keys: dict = {K_w: 0.0, K_a: 0.0, K_s: 0.0, K_d: 0.0}
        strength: list

        for key in keys:
            keys[key] = 1.0 if is_pressed[key] else 0.0

        strength = [keys[K_d] - keys[K_a], keys[K_s] - keys[K_w]]
        strength_norm = norm(strength)

        if strength_norm:
            strength /= strength_norm

        return strength


In [ ]:
class Entity:
    position: list
    color: tuple = (115, 10, 46)

    def _draw(self):
        RECT_SIZE: int = 40
        HALF_SIZE: int = RECT_SIZE // 2  # Desloca a âncora para o centro
        RECT_TRANSFORM: tuple = (self.position[0] - HALF_SIZE,
                                 self.position[1] - HALF_SIZE, RECT_SIZE, RECT_SIZE)

        return pygame.draw.rect(screen, self.color, RECT_TRANSFORM)

    def get_x(self) -> int:
        return self.position[0]

    def get_y(self) -> int:
        return self.position[1]

    def __init__(self, coords: tuple) -> None:
        self.position = list(coords)


In [ ]:
class KinematicBody(Entity):
    speed: float = 1.0
    velocity: list = [0.0, 0.0]

    def move(self) -> None:

        for i in range(2):
            self.position[i] += self.velocity[i] * self.speed

            if self.position[i] < 0.0:
                self.position[i] = 0.0
            elif self.position[i] > SCREEN_SIZE[i]:
                self.position[i] = SCREEN_SIZE[i]

    def _input(self) -> None:
        self.velocity = InputEvent.get_input_strength()

    # def _input_event(self, event: InputEvent) -> None:
    #     pass

    def __init__(self, coords: tuple, color=(46, 10, 115)) -> None:
        super().__init__(coords)
        self.color = color


In [ ]:
body: KinematicBody = KinematicBody((WIDTH // 2, HEIGHT // 2))
entity: Entity = Entity((randint(0, WIDTH), randint(0, HEIGHT)))
COLOR_BLACK: tuple = (0, 0, 0)

while True:
    clock.tick(FIXED_FPS)
    screen.fill(COLOR_BLACK)

    for event in pygame.event.get():
        if event.type == QUIT:
            pygame.quit()
            exit()

#         if event.type == KEYDOWN:
#             keys: list = [K_w, K_a, K_s, K_d]
#
#             for key in keys:
#                 if event.key == key:
#                     body._input_event(
#                         InputEvent.InputTypes.JUST_PRESSED, key)

    body._input()
    body.move()
    body_aabb = body._draw()
    entity_aabb = entity._draw()
    
    if body_aabb.colliderect(entity_aabb):
        entity.position = (randint(0, WIDTH), randint(0, HEIGHT))
    
    pygame.display.update()
